## Experiments

Let's start playing around with our data

In [36]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import nltk
nltk.download('punkt')
from sklearn.feature_extraction import DictVectorizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package punkt to /home/jonat/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Read in training data
This dataset contains 100 annotated terms of service contracts, each row represents a sentence, which carries on it a label. The label corresponds to a different type of potential unfairness, as defined by the authors of CLAUDETTE, the previous paper from which this dataset came from. 

In [37]:
df = pd.read_csv('../data/dataset.csv')
df.head()


,Unnamed: 0,A,CH,CR,J,LAW,LTD,PINC,TER,USE,document,document_ID,label,text,TER_targets,LTD_targets,A_targets,CH_targets,CR_targets
0,0,0,0,0,0,0,0,0,0,0,Mozilla,0,0,websites & communications terms of use,NaN,NaN,NaN,NaN,NaN
1,1,0,0,0,0,0,0,0,0,0,Mozilla,0,0,please read the terms of this entire document ...,NaN,NaN,NaN,NaN,NaN
2,2,0,0,0,0,0,0,0,0,1,Mozilla,0,1,by accessing or signing up to receive communic...,NaN,NaN,NaN,NaN,NaN
3,3,0,0,0,0,0,0,0,0,0,Mozilla,0,0,our websites include multiple domains such as ...,NaN,NaN,NaN,NaN,NaN
4,4,0,0,0,0,0,0,0,0,0,Mozilla,0,0,you may also recognize our websites by nicknam...,NaN,NaN,NaN,NaN,NaN


get the vocabulary for the dataset

In [ ]:
# from datasets import load_dataset
# #load in the pile of law dataset
# pol = load_dataset("pile-of-law/pile-of-law", 'tos',split="train + validation")

Now let's load in a pretrained huggingface BERT model  (https://huggingface.co/pile-of-law/legalbert-large-1.7M-2) to get the word embeddings of each sentence

In [69]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('pile-of-law/legalbert-large-1.7M-2')
model = BertModel.from_pretrained('pile-of-law/legalbert-large-1.7M-2')
model.to(device)
text = "Replace me by any text you'd like.".split()
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [70]:
vocab = tokenizer.get_vocab()
vocab
tokenizer.encode_plus("hello world", return_tensors='pt')

{'input_ids': tensor([[    2, 19698,   515,  3956,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [38]:
model.device

device(type='cuda', index=0)

The pre trained model gives encodings of shape 12 x 1024 for a sentence, this will be useful to know when we use it as part of our model.

Now Let's get embeddings of the sentences in our data

In [42]:
vec = DictVectorizer()
df['text'] = df['text'].str.split()
df['text']

0            [websites, &, communications, terms, of, use]
1        [please, read, the, terms, of, this, entire, d...
2        [by, accessing, or, signing, up, to, receive, ...
3        [our, websites, include, multiple, domains, su...
4        [you, may, also, recognize, our, websites, by,...
                               ...                        
20412    [you, may, terminate, your, account, at, our, ...
20413    [the, following, provisions, shall, survive, t...
20414    [the, company, reserves, the, right, ,, at, it...
20415    [your, continued, use, of, the, service, ,, fo...
20416    [the, last, revision, will, be, reflected, in,...
Name: text, Length: 20417, dtype: object

In [11]:
def embed_text(text,tokenizer=BertTokenizer.from_pretrained('pile-of-law/legalbert-large-1.7M-2'),model=BertModel.from_pretrained('pile-of-law/legalbert-large-1.7M-2'),device=device):
    # text = tensor of one hot encoding of text .to(device)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    return output.last_hidden_state[0]

df["embeddings"] = df["text"].apply(embed_text)
    

ValueError: too many dimensions 'str'